<a href="https://colab.research.google.com/github/azwanzuharimi/Malaysian-Pay-Gap-Cleaning-Sheets-to-CSV-/blob/main/Malaysian_Pay_Gap_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from google.colab import auth
import pandas as pd
import gspread
from google.auth import default
import numpy as np 

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('Malaysian Pay Gap: Data Submission (Responses) - Public').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

# Convert to a DataFrame and render.

df = pd.DataFrame.from_records(rows)

# Cleaning data and columns
cols = rows[0]
df.columns = cols
df = df.iloc[1:,:]

# Transform column names as there are duplicated column names
col_list = []

for x in df.columns:
  if x not in col_list:
    col_list.append(x)
  else:
    col_list.append(x + '_others')

df.columns = col_list

# Change data types
for x in col_list[1:]:
  try:
    df[x] = pd.to_numeric(df[x])
  except:
    continue

df['Timestamp'] = pd.to_datetime(df['Timestamp'], 'coerce')
df = df.replace("", np.nan)
df = df.replace(" ", np.nan)
display(df.head())
print(df.shape)

,Timestamp,Age,Race,Gender,Nationality,Qualification,Job Position,Exp,# Jobs,Malaysia,...,Industry_others,Specialisation,Specialisation_others,Work hours,Work days,Current Salary,First Job Salary,Compensation Satisfaction,Overall Satisfaction,Comments
1,2022-03-12 19:48:40,37.0,Chinese,Female,Malaysia,Master's Degree (Overseas),Associate,12.0,4,Yes,...,NaN,Architect,NaN,10.0,5.0,18000.0,4500.0,4,4,Have confidence and do not undercut your own s...
2,2022-03-12 19:30:38,28.0,Malay,Female,Malaysia,Bachelor's Degree,Executive Customer Experience,5.0,2,Yes,...,NaN,Customer Service,NaN,8.0,5.0,3500.0,2400.0,4,3,NaN
3,2022-03-12 19:21:08,30.0,Malay,Male,Malaysia,Bachelor's Degree,Fund Accounting Analyst ll,5.0,5,Yes,...,NaN,Accounting,NaN,9.0,5.0,6575.0,3400.0,3,3,NaN
4,2022-03-12 19:18:06,33.0,Malay,Male,Malaysia,Bachelor's Degree,Mill engineer,9.0,2,Yes,...,NaN,Engineering,NaN,10.0,6.0,4500.0,2600.0,2,3,NaN
5,2022-03-12 19:10:28,27.0,Malay,Female,Malaysia,Bachelor's Degree,Site Quantity Surveyor,2.5,5,Yes,...,NaN,Quantity Survey,NaN,8.0,6.0,3000.0,2000.0,2,4,NaN


(2782, 24)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782 entries, 1 to 2782
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Timestamp                  2782 non-null   object 
 1   Age                        2765 non-null   float64
 2   Race                       2782 non-null   object 
 3   Gender                     2782 non-null   object 
 4   Nationality                2782 non-null   object 
 5   Qualification              2782 non-null   object 
 6   Job Position               2782 non-null   object 
 7   Exp                        2782 non-null   float64
 8   # Jobs                     2782 non-null   int64  
 9   Malaysia                   2782 non-null   object 
 10  Remote                     2782 non-null   object 
 11  State                      2782 non-null   object 
 12  Type of Company            2782 non-null   object 
 13  Industry                   2782 non-null   objec

In [50]:
# NOTE: To_excel instead to csv because somehow, writing to CSV file causes error in UTF-8 encoding (I'm is shown as Iâ€™m)
df.to_excel('sal.xlsx', index=False, encoding='utf-8')

In [49]:
# df.to_csv('test.csv', index=False, encoding='utf-8')